In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.layers import RepeatVector
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, confusion_matrix

In [ ]:
# Load the dataset from CSV file
data = pd.read_csv('dataset.csv')
data.head(135)

In [5]:
# Convert the python code strings into a numerical representation using one-hot encoding
enc = OneHotEncoder()
enc.fit(data)
df = enc.transform(data)

In [6]:
# Split the dataset into training and testing sets
train_size = int(df.shape[0] * 0.8)
X = df[:, :-1]
y = df[:, -1].toarray().ravel()
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
X_train = X_train.toarray()
X_test = X_test.toarray()
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [7]:
# Define the Encoder-Decoder LSTM model
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
encoder = LSTM(128)(inputs)
decoder = RepeatVector(X_train.shape[1])(encoder)
decoder = LSTM(128, return_sequences=True)(decoder)
outputs = Dense(X_train.shape[2], activation='softmax')(decoder)
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model on the training set
history = model.fit(X_train, X_train, epochs=90, validation_data=(X_test, X_test), batch_size=28)

Epoch 1/90
4/4 [==============================] - 10s 1s/step - loss: 0.9923 - val_loss: 0.9923
Epoch 2/90
4/4 [==============================] - 3s 754ms/step - loss: 0.9923 - val_loss: 0.9923
Epoch 3/90
4/4 [==============================] - 3s 755ms/step - loss: 0.9923 - val_loss: 0.9923
Epoch 4/90
4/4 [==============================] - 3s 813ms/step - loss: 0.9923 - val_loss: 0.9923
Epoch 5/90
4/4 [==============================] - 3s 839ms/step - loss: 0.9923 - val_loss: 0.9923
Epoch 6/90
4/4 [==============================] - 3s 781ms/step - loss: 0.9923 - val_loss: 0.9923
Epoch 7/90
4/4 [==============================] - 3s 772ms/step - loss: 0.9923 - val_loss: 0.9923
Epoch 8/90
4/4 [==============================] - 3s 779ms/step - loss: 0.9923 - val_loss: 0.9923
Epoch 9/90
4/4 [==============================] - 3s 819ms/step - loss: 0.9923 - val_loss: 0.9923
Epoch 10/90
4/4 [==============================] - 4s 900ms/step - loss: 0.9923 - val_loss: 0.9923
Epoch 11/90
4/4 [====

Epoch 84/90
4/4 [==============================] - 4s 1s/step - loss: 0.9923 - val_loss: 0.9923
Epoch 85/90
4/4 [==============================] - 4s 1s/step - loss: 0.9923 - val_loss: 0.9923
Epoch 86/90
4/4 [==============================] - ETA: 0s - loss: 0.9923

In [ ]:
# Evaluate the performance of the model on the testing set
from sklearn.metrics import accuracy_score
y_pred = np.argmax(model.predict(X_test), axis=1)
loss = model.evaluate(X_test, X_test)
print("Test loss:", loss)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy : ", accuracy)

f1 = f1_score(y_test, y_pred, average='macro')
print("F1 score", f1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(y_test,'b',label='Original')
plt.plot(y_pred,'r',label='Predicted')
plt.title('Predicted and Original Values')
plt.xlabel('Original')
plt.ylabel('Predicted')
plt.legend()
plt.show()

In [ ]:
# Plot the training loss as a function of the number of epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
f1 = f1_score(y_test, y_pred, average='macro')
print("F1 score", f1)
precision = precision_score(y_test, y_pred, average='macro')
print("Precision score", precision)
recall = recall_score(y_test, y_pred, average='macro')
print("Recall score", recall)

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix: \n", cm)